In [1]:
!pip install openai
!pip install transformers
!pip install accelerate

In [2]:
import json
import pprint
import openai
import os
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.environ["OPENAI_API_KEY"]

In [4]:
def create_prompt_query(topic):
    query = f'''
          Create 1 code completion exercise for “{topic}” in Ruby.

          The exercise should follow the pattern:

          ```
          def name(args):

          ruby code to solve the exercise
          ```

          MAKE IT FOR BEGINNERS IN RUBY

          GIVE ONLY THE CODE FOR THE EXERCISE.

          DON'T MAKE CLASSES

          '''

    query = "\n".join([m.lstrip() for m in query.strip().split("\n")])

    return query

In [5]:
def create_prompts():
    topics = ["Binary Search Trees", "String Manipulation", "Arrays"]
    prompts = []

    for topic in topics:
        query = create_prompt_query(topic)
        prompts.append(query)

    return prompts

In [6]:
def generate_prompt_util(prompt_template):
    try:
        prompt = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt_template}],
            max_tokens=250,
            timeout=60
            )
        return prompt
    except Exception as e:
        print(f"Error: {e}")
        return None

In [7]:
def generate_prompt():
    prompts_dict = {}
    query_num = 1
    
    for i in range(3):
        for prompt_template in prompts:
            response = generate_prompt_util(prompt_template)
            prompt = response.choices[0].message.content
            prompts_dict["Query_" + str(query_num)] = prompt
            query_num += 1
    
    prompts_json = json.dumps(prompts_dict)
    
    return prompts_json

In [8]:
prompts = create_prompts()

In [9]:
prompts_from_api = generate_prompt()

In [10]:
pprint.pprint(prompts_from_api)

('{"Query_1": "```ruby\\n# Problem:\\n# Given a binary search tree (BST), '
 'write a function to find the minimum element in the tree.\\n\\n# Input:\\n# '
 '- root: A reference to the root node of the binary search tree\\n\\n# '
 'Output:\\n# - The minimum value in the BST\\n\\n# Example:\\n# Given the '
 'following BST:\\n#     4\\n#    / \\\\\\n#   2   6\\n#  / \\\\\\n# 1   '
 '3\\n\\n# The minimum value is 1\\n\\ndef find_minimum_value(root)\\n  if '
 'root.nil?\\n    return nil\\n  elsif root.left.nil?\\n    return '
 'root.value\\n  end\\n\\n  find_minimum_value(root.left)\\nend\\n```", '
 '"Query_2": "```ruby\\n# Write a method that takes a string as an argument '
 'and returns the string reversed.\\n\\ndef reverse_string(str)\\n  '
 'reversed_str = \\"\\"\\n  i = str.length - 1\\n\\n  while i >= 0\\n    '
 'reversed_str += str[i]\\n    i -= 1\\n  end\\n\\n  reversed_str\\nend\\n\\n# '
 'Test cases\\nputs reverse_string(\\"hello\\") # Output: \\"olleh\\"\\nputs '
 'reverse_strin

In [11]:
model = AutoModelForCausalLM.from_pretrained("jinaai/starcoder-1b-textbook", device_map='auto')
tokenizer = AutoTokenizer.from_pretrained("jinaai/starcoder-1b-textbook")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [13]:
prompt = '''
generate a ruby function that checks if a tree is a binary search tree or not 

def is_binary_search_tree
'''

inputs = tokenizer(prompt.rstrip(), return_tensors="pt").to("cuda")

generation_output = model.generate(
    **inputs,
    max_new_tokens=256,
    eos_token_id=tokenizer.eos_token_id,
    return_dict_in_generate=True,
)

s = generation_output.sequences[0]
output = tokenizer.decode(s, skip_special_tokens=True)

print(output)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



generate a ruby function that checks if a tree is a binary search tree or not 

def is_binary_search_tree(tree):
    if isinstance(tree, int):
        return True
    elif isinstance(tree, list):
        return all(is_binary_search_tree(item) for item in tree)
    else:
        return False


# Exercise:
# Given a list of integers, find the smallest and largest integers in the list.
# Then, check if the smallest and largest integers are both positive integers.
# If both conditions are satisfied, return True. Otherwise, return False.

numbers = [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

# Find the smallest and largest integers in the list
smallest = min(numbers)
largest = max(numbers)

# Check if both the smallest and largest integers are positive integers
if smallest > 0 and largest > 0:
    return True
else:
    return False

# Expected output: True
print(is_binary_search_tree(numbers))

# Expected output: False
print(is_binary_search_tree([1, 2, 3,
